In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
import matplotlib
import tensorly as tl
import numpy as np
import json
import time
import random
from datetime import datetime
from tensorly.decomposition import non_negative_tucker
import collections
from pandas import Series
from urllib.request import urlopen,quote

In [2]:
#删除只有一次出行车辆
def shanchu_yicichuxing(df):
    #统计拍摄次数
    tongji=Series(collections.Counter(df.index)).to_frame().rename(columns={0:'num'})
    #拍摄次数大于1的车辆id
    carid=tongji[tongji['num']>1].index
    #保留拍摄次数大于1的车辆
    return df.loc[carid]

In [3]:
#每辆非运营车通过两卡口之间的时间差，需要运行10mins
def shijiancha(df):
    df=df.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
    #print(df)
    kakou_shijiancha=[]
    chushi_carid=-1
    chushi_time=-1
    #a=0#计数
    for index, row in df.iterrows():
        #print(a)
        #a=a+1
        if index!=chushi_carid:
            chushi_carid=index
            chushi_time=row['time']
        else:
            kakou_shijiancha.append(row['time']-chushi_time)
            chushi_time=row['time']
            #print(kakou_shijiancha)
    #print(kakou_shijiancha)
    return kakou_shijiancha

In [4]:
#计算上下四分位数,lis按从小到大排过序
def count_quartiles(lis):
    s=pd.Series(lis)
    q1=s.quantile(q=0.25)
    q3=s.quantile(q=0.75)
    return q1,q3

In [5]:
#计算上下边缘,lis按从小到大排过序
def count_margin(q1, q3, a):
    iqr = q3 - q1
    upper = q3 + a*iqr
    floor = q1 - a*iqr
    return upper, floor

In [6]:
#切割一日出行
def qiege(df, upper, floor):
    df=df.sort_values(ascending=True,by=['time'])#时间从小到大排序
    qiege_df=[] #df切割后放入qiege_df
    chushi_df=pd.DataFrame(columns=df.columns)
    chushi_df.index.name='car_id'
    chushi_time=-1
    i=0
    for index, row in df.iterrows():
        i=i+1
        if i==1:
            chushi_time=row['time']
            chushi_df=chushi_df.append(row)
        else:#
            if row['time']-chushi_time>=upper:
                qiege_df.append(chushi_df)
                chushi_df=pd.DataFrame(columns=df.columns)
                chushi_df.index.name='car_id'
                chushi_df=chushi_df.append(row)
            elif row['time']-chushi_time<=floor:
                chushi_df=chushi_df.append(row)
            else:
                chushi_df=chushi_df.append(row) 
            chushi_time=row['time']
        if i==len(df):
            qiege_df.append(chushi_df)
    return qiege_df

In [7]:
#构建（起点卡口，终点卡口，出发时刻，到达时刻）为一个节点
def goujian_jiedian(df,upper, floor):
    df=df.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
    l_jiedian = []
    #jiedian = pd.DataFrame(columns=['start_kid','end_kid','start_time','end_time']) 
    car_ids=list(set(df.index))
    n=0
    for car_id in car_ids:
        n=n+1
        #print(n)
        one_feiyunying=df.loc[car_id]
        qiege_dfs=qiege(one_feiyunying,upper, floor)
        for qiege_df in qiege_dfs:
            if len(qiege_df)>1:
                #print(qiege_df)
                if qiege_df.iloc[0]['id']!=qiege_df.iloc[len(qiege_df)-1]['id']:
                #print(qiege_df)
                    qiege_df=qiege_df.sort_values(ascending=True,by=['car_id','time'])#时间从小到大排序
                    l_jiedian.append([qiege_df.iloc[0]['id'],qiege_df.iloc[len(qiege_df)-1]['id'],
                                   qiege_df.iloc[0]['time'],qiege_df.iloc[len(qiege_df)-1]['time'],str(car_id)])
                    #print(qiege_df.iloc[0]['id'])
                    #print(qiege_df.iloc[len(qiege_df)-1]['id'])
                    #print(qiege_df.iloc[0]['time'])
                    #print(qiege_df.iloc[len(qiege_df)-1]['time'])
                    #print(type(str(car_id)))
                    #print(str(car_id))
                    '''new_line=pd.Series({'start_kid':qiege_df.iloc[0]['id'],
                                        'end_kid':qiege_df.iloc[len(qiege_df)-1]['id'],
                                        'start_time':qiege_df.iloc[0]['time'],
                                        'end_time':qiege_df.iloc[len(qiege_df)-1]['time']}) '''         
                    #jiedian=jiedian.append(new_line,ignore_index=True)
                    #print(jiedian)
    jiedian = pd.DataFrame(l_jiedian,columns=['start_kid','end_kid','start_time','end_time','car_id'])
    #print(jiedian)
    return jiedian

In [8]:
'''
feiyunyingche_1205=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205=shanchu_yicichuxing(feiyunyingche_1205)
list_kakou_shijiancha=shijiancha(feiyunyingche_1205)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1205,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_jiedian.csv',index=False)
'''

"\nfeiyunyingche_1205=pd.read_csv(open(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181\x8161205_title_feiyunyingche_zuobiao.csv'),index_col='car_id') \nfeiyunyingche_1205=shanchu_yicichuxing(feiyunyingche_1205)\nlist_kakou_shijiancha=shijiancha(feiyunyingche_1205)\nlist_kakou_shijiancha.sort()#升序\nq1, q3=count_quartiles(list_kakou_shijiancha)\nupper, floor=count_margin(q1, q3, 1.6)\nprint(len(list_kakou_shijiancha))\nprint(q1, q3)\nprint(upper, floor)\n#运行4hours\ngoujian_jiedian(feiyunyingche_1205,upper, floor).to_csv(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181P5_jiedian.csv',index=False)\n"

In [10]:
'''
feiyunyingche_1210=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1210=shanchu_yicichuxing(feiyunyingche_1210)
list_kakou_shijiancha=shijiancha(feiyunyingche_1210)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1210,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_jiedian.csv',index=False)
'''

"\nfeiyunyingche_1210=pd.read_csv(open(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181\x8161210_title_feiyunyingche_zuobiao.csv'),index_col='car_id') \nfeiyunyingche_1210=shanchu_yicichuxing(feiyunyingche_1210)\nlist_kakou_shijiancha=shijiancha(feiyunyingche_1210)\nlist_kakou_shijiancha.sort()#升序\nq1, q3=count_quartiles(list_kakou_shijiancha)\nupper, floor=count_margin(q1, q3, 1.6)\nprint(len(list_kakou_shijiancha))\nprint(q1, q3)\nprint(upper, floor)\n#运行4hours\ngoujian_jiedian(feiyunyingche_1210,upper, floor).to_csv(r'F:\\运营车识别\\王宇坤 出租车识别\x819.12.181Q0_jiedian.csv',index=False)\n"

In [8]:
feiyunyingche_1205_1=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_class1_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205_1=shanchu_yicichuxing(feiyunyingche_1205_1)
list_kakou_shijiancha=shijiancha(feiyunyingche_1205_1)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1205_1,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_1_jiedian.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


719928
292.0 10202.0
26058.0 -15564.0


In [9]:
feiyunyingche_1205_2=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_class2_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205_2=shanchu_yicichuxing(feiyunyingche_1205_2)
list_kakou_shijiancha=shijiancha(feiyunyingche_1205_2)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1205_2,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_2_jiedian.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1843944
207.0 2159.0
5282.200000000001 -2916.2000000000003


In [10]:
feiyunyingche_1205_3=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_class3_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205_3=shanchu_yicichuxing(feiyunyingche_1205_3)
list_kakou_shijiancha=shijiancha(feiyunyingche_1205_3)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1205_3,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_3_jiedian.csv',index=False)

712080
219.0 8089.0
20681.0 -12373.0


In [11]:
feiyunyingche_1205_4=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_class4_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1205_4=shanchu_yicichuxing(feiyunyingche_1205_4)
list_kakou_shijiancha=shijiancha(feiyunyingche_1205_4)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1205_4,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1205_4_jiedian.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1831699
192.0 1988.0
4861.6 -2681.6000000000004


In [12]:
feiyunyingche_1210_1=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_class1_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1210_1=shanchu_yicichuxing(feiyunyingche_1210_1)
list_kakou_shijiancha=shijiancha(feiyunyingche_1210_1)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1210_1,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_1_jiedian.csv',index=False)

546357
290.0 9246.0
23575.6 -14039.6


In [13]:
feiyunyingche_1210_2=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_class2_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1210_2=shanchu_yicichuxing(feiyunyingche_1210_2)
list_kakou_shijiancha=shijiancha(feiyunyingche_1210_2)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1210_2,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_2_jiedian.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1381935
214.0 2184.0
5336.0 -2938.0


In [14]:
feiyunyingche_1210_3=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_class3_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1210_3=shanchu_yicichuxing(feiyunyingche_1210_3)
list_kakou_shijiancha=shijiancha(feiyunyingche_1210_3)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1210_3,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_3_jiedian.csv',index=False)

D:\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1454157
199.0 1942.0
4730.8 -2589.8


In [15]:
feiyunyingche_1210_4=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_class4_zuobiao.csv'),index_col='car_id') 
feiyunyingche_1210_4=shanchu_yicichuxing(feiyunyingche_1210_4)
list_kakou_shijiancha=shijiancha(feiyunyingche_1210_4)
list_kakou_shijiancha.sort()#升序
q1, q3=count_quartiles(list_kakou_shijiancha)
upper, floor=count_margin(q1, q3, 1.6)
print(len(list_kakou_shijiancha))
print(q1, q3)
print(upper, floor)
#运行4hours
goujian_jiedian(feiyunyingche_1210_4,upper, floor).to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\1210_4_jiedian.csv',index=False)

554149
226.0 7241.0
18465.0 -10998.0


# 调用百度api计算两个卡口之间的驾车时间

In [ ]:
'''
#调用百度api返回两点之间所需时间（单位为秒）
#ak:lWpbR5OCQYybppqci2kGYgFd
#ak:ceSRRXaqrZRffmNThRZzgEohUSiiZkVT
#ak:ogSom1tX1Yuc6ffl87vxuQNaqH3n8dtG
def return_shijian(origin_kid_x,origin_kid_y,des_kid_x,des_kid_y):
    url = 'http://api.map.baidu.com/direction/v2/driving?'
    url=url+'origin='+str(origin_kid_y)+','+str(origin_kid_x)+'&destination='+str(des_kid_y)+','+str(des_kid_x)+'&coord_type=wgs84&tactics=2&ak='+'lWpbR5OCQYybppqci2kGYgFd'        
    #print(url)
    req = urlopen(url)
    res  = req.read().decode()
    if json.loads(res)['status']==0:#成功返回
        #print(1)
        shijian = json.loads(res)['result']['routes'][0]['duration']
    else:
        #print(json.loads(res)['status'])
        shijian=-1
    return shijian
'''

In [ ]:
'''
feiyunyingche_1205=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161205_title_feiyunyingche_zuobiao.csv'),index_col='car_id')
feiyunyingche_1210=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\20161210_title_feiyunyingche_zuobiao.csv'),index_col='car_id') 
kid_baidu=list(set(list(set(feiyunyingche_1205['id']))+
                   list(set(feiyunyingche_1210['id']))))


kakou_time = pd.DataFrame(columns=['origin_kid','des_kid','time']) 
for i in range(len(kid_baidu)):
    for j in range(len(kid_baidu)):
        #print(i,j)
        if i!=j:  #其实应该有i==j，只不过时间为0
            new_line=pd.Series({'origin_kid':kid_baidu[i],
                                'des_kid':kid_baidu[j],
                                'time':-1})
            kakou_time=kakou_time.append(new_line,ignore_index=True)
kakou_time.to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv',index=False)

new_kakou_location_m=pd.read_csv(open(r'F:\运营车识别\new_kakou_location_m.csv'))

#反复运行，直到全部都有时间
kakou_time=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv'))
for index, row in kakou_time.iterrows():
    if row['time']==-1:
        origin_kid_x=new_kakou_location_m[new_kakou_location_m['k_id']==row['origin_kid']]['x'].iloc[0]
        origin_kid_y=new_kakou_location_m[new_kakou_location_m['k_id']==row['origin_kid']]['y'].iloc[0]
        des_kid_x=new_kakou_location_m[new_kakou_location_m['k_id']==row['des_kid']]['x'].iloc[0]
        des_kid_y=new_kakou_location_m[new_kakou_location_m['k_id']==row['des_kid']]['y'].iloc[0]
        #print(origin_kid_x)
        #print(origin_kid_y)
        #print(des_kid_x)
        #print(des_kid_y)
        shijian=return_shijian(origin_kid_x,origin_kid_y,des_kid_x,des_kid_y)
        print(shijian)
        if shijian!=-1:
            kakou_time.loc[index,'time']=shijian
        else:
            break
#print(kakou_time)
kakou_time.to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv',index=False)
'''

In [20]:
'''
kakou_time=pd.read_csv(open(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv'))
kid_baidu=list(set(list(set(kakou_time['origin_kid']))+
                   list(set(kakou_time['des_kid']))))
for i in range(len(kid_baidu)):
    for j in range(len(kid_baidu)):
        if i==j:
            new_line=pd.Series({'origin_kid':kid_baidu[i],
                                    'des_kid':kid_baidu[j],
                                    'time':0})
            kakou_time=kakou_time.append(new_line,ignore_index=True)
kakou_time.to_csv(r'F:\运营车识别\王宇坤 出租车识别\2019.12.181\kakou_time.csv',index=False)
'''